In [2]:
#facec detection
import numpy as np
import cv2
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height
while(True):
    ret, frame = cap.read()
    #frame = cv2.flip(frame, -1) # Flip camera vertically
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
   # cv2.imshow('gray', gray)
    
    
    #frame = cv2.flip(frame, -1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 2)
    #faces = faceCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5,minSize=(22, 22))
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
   
    cv2.imshow('video',frame)
    k = cv2.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to quit
        break

k = cv2.waitKey(0)
if k == 27:         # wait for ESC key to exit
    cv2.destroyAllWindows()
elif k == ord('s'): # wait for 's' key to save and exit
    
    cv2.destroyAllWindows()
cap.release()

In [10]:
#face dataset
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 22 17:50:44 2021

@author: sohai
"""

#import numpy as np

import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

id = input('enter user id')

sampleN=0;

while 1:

    ret, img = cap.read()

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:

        sampleN=sampleN+1;

        cv2.imwrite("dataset/"+str(id)+ "." +str(sampleN)+ ".jpg", gray[y:y+h, x:x+w])

        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

        cv2.waitKey(100)

    cv2.imshow('img',img)

    cv2.waitKey(1)

    if sampleN > 20:

        break

cap.release()

cv2.destroyAllWindows()

In [8]:
#face traning
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 22 16:39:14 2021

@author: sohai
"""

import cv2
import os
import numpy as np
from PIL import Image



# Path for face image database
path = 'dataset'
recognizer = cv2.face.LBPHFaceRecognizer_create() 
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

# function to get the images and label data
def getImagesAndLabels(path):

    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    print(imagePaths)
    for imagePath in imagePaths:

        PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        img_numpy = np.array(PIL_img,'uint8')
        Id = int(os.path.split(imagePath)[-1].split(".")[0])
        faces = detector.detectMultiScale(img_numpy)

        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(Id)

    return faceSamples,ids

print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))
#labels = np.random.randint(1, size=len(faces))
#recognizer.train(faces, np.array(labels))

# Save the model into trainer/trainer.yml
recognizer.save('trainer/trainer.yml') # recognizer.save() worked on Mac, but not on Pi

# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))


 [INFO] Training faces. It will take a few seconds. Wait ...
['dataset\\1.1.jpg', 'dataset\\1.10.jpg', 'dataset\\1.2.jpg', 'dataset\\1.3.jpg', 'dataset\\1.4.jpg', 'dataset\\1.5.jpg', 'dataset\\1.6.jpg', 'dataset\\1.7.jpg', 'dataset\\1.8.jpg', 'dataset\\1.9.jpg', 'dataset\\12.1.jpg', 'dataset\\12.10.jpg', 'dataset\\12.2.jpg', 'dataset\\12.3.jpg', 'dataset\\12.4.jpg', 'dataset\\12.5.jpg', 'dataset\\12.6.jpg', 'dataset\\12.7.jpg', 'dataset\\12.8.jpg', 'dataset\\12.9.jpg', 'dataset\\2.1.jpg', 'dataset\\2.10.jpg', 'dataset\\2.2.jpg', 'dataset\\2.3.jpg', 'dataset\\2.4.jpg', 'dataset\\2.5.jpg', 'dataset\\2.6.jpg', 'dataset\\2.7.jpg', 'dataset\\2.8.jpg', 'dataset\\2.9.jpg', 'dataset\\sudais.1.jpg', 'dataset\\sudais.10.jpg', 'dataset\\sudais.2.jpg', 'dataset\\sudais.3.jpg', 'dataset\\sudais.4.jpg', 'dataset\\sudais.5.jpg', 'dataset\\sudais.6.jpg', 'dataset\\sudais.7.jpg', 'dataset\\sudais.8.jpg', 'dataset\\sudais.9.jpg']


ValueError: invalid literal for int() with base 10: 'sudais'

In [1]:
#face recogination
import cv2
import numpy as np
import os 
import requests
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
import smtplib
import math
import random
from twilio.rest import Client


recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('recognizer/trainningData.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);
digits="0123456789"
OTP=""

font = cv2.FONT_HERSHEY_SIMPLEX
fromaddr = "sohailkarimkhugno@gmail.com"
password = "lmgwjkkdirjgtpxl"
receiver = "sohail.karimsk65@gmail.com"  
cam = cv2.VideoCapture(0)
def SendMail(dataset):
    img_data = open(dataset, 'rb').read()
    msg = MIMEMultipart()
    msg['Subject'] = 'subject'
    msg['From'] = fromaddr
    msg['To'] = receiver

    text = MIMEText('text for email')
    msg.attach(text)
    image = MIMEImage(img_data, name=os.path.basename(dataset))
    msg.attach(image)

    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login(fromaddr, password)
    s.sendmail(fromaddr,receiver, msg.as_string())
    print("Mail send successfully.")
    s.quit()
#iniciate id counter
id = 0

# names related to ids: example ==> Marcelo: id=1,  etc
names = ['none' , 'sohail' ,'sohail' ,'sajid','meherban' ] 

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
#for sending opt 
account_sid = 'AC412b3b3fa8587df789d41116da99b3a4'
auth_token = '73fc903ec7a7485591daea3a7c1a1f60'
client = Client(account_sid, auth_token)

while True:

    ret, img =cam.read()
    #img = cv2.flip(img, -1) # Flip vertically

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )

    for(x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        # Check if confidence is less them 100 ==> "0" is perfect match 
        if (confidence > 40):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
            #dimg= cam.read()[1]
            cv2.imwrite( "image.jpg",img)       
            SendMail('image.jpg')

            #sending otp
            for i in range(6):
                OTP+=digits[math.floor(random.random()*10)]
            otp = OTP 
            message = client.messages.create(
                body='The vericification code for ATM transaction is' '\n' +  otp  + '\n'  'please dont share with others',
                from_='+19035322880',
                to='+923420496364'
             )
            print("Opt is sent")
            #otp verification     
            a = input("Enter your otp")
            if a == OTP:
                print("otp is verify")

            else:
                print("pease check your opt again")
                
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
    
    cv2.imshow('cam',img) 

    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()





## for caputring image and sending it through email.


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\face\src\facerec.cpp:61: error: (-2:Unspecified error) File can't be opened for reading! in function 'cv::face::FaceRecognizer::read'


In [1]:
 #email code
import smtplib
from email import encoders
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage


gmail_user = "sohailkarimkhugno@gmail.com"
gmail_pwd = "smlfzztgitpnswms"

to = "sohail.karimsk65@gmail.com"
subject = "Report"
text = "Picture report from Allied ATM"
attach = 's.jpg'

msg = MIMEMultipart()

msg['From'] = gmail_user
msg['To'] = to
msg['Subject'] = subject

msg.attach(MIMEText(text))

part = MIMEBase('application', 'octet-stream')
part.set_payload(open(attach, 'rb').read())
encoders.encode_base64(part)
part.add_header('Content-Disposition',
   'attachment; filename="%s"' % os.path.basename(attach))
msg.attach(part)

mailServer = smtplib.SMTP("smtp.gmail.com", 587)
mailServer.ehlo()
mailServer.starttls()
mailServer.ehlo()
mailServer.login(gmail_user, gmail_pwd)
mailServer.sendmail(gmail_user, to, msg.as_string())
# Should be mailServer.quit(), but that crashes...
mailServer.close()
print("The email is sent : ")

SMTPServerDisconnected: Server not connected

In [16]:
#otp message
import math
import random
import smtplib
import os
from twilio.rest import Client
digits="0123456789"
OTP=""


# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = 'AC412b3b3fa8587df789d41116da99b3a4'
auth_token = '73fc903ec7a7485591daea3a7c1a1f60'
client = Client(account_sid, auth_token)
for i in range(6):
    OTP+=digits[math.floor(random.random()*10)]
otp = OTP 
message = client.messages.create(
         body='The vericification code for ATM transaction is' '\n' +  otp  + '\n'  'please dont share with others',
         from_='+19035322880',
         to='+923420496364'
     )
print("Opt is sent")
#otp verification     
a = input("Enter your otp")
if a == OTP:
    print("otp is verify")

else:
   print("pease check your opt again")
print(message.sid)

pease check your opt again
SMc115d37c548e4776bde67457e32f3d33
